# Agentic RAG

In [32]:
!pip install smolagents
!pip install langchain
!pip install langchain-community
!pip install duckduckgo-search
!pip install bs4
!pip install requests

In [23]:
from google.colab import userdata as ud
from huggingface_hub import login

env_token = ud.get('HF_TOKEN')
login(env_token)

In [33]:
from langchain_community.tools import DuckDuckGoSearchRun
import os

search_tool = DuckDuckGoSearchRun()

print(search_tool.invoke("Who wrote odyssey poem?"))

In antiquity, Homer's authorship was taken as true, but contemporary scholarship predominantly assumes that the Iliad and the Odyssey were composed independently, forming as part of long oral traditions. Given widespread illiteracy, the poem was performed for an audience by an aoidos or rhapsode. Odyssey, epic poem in 24 books traditionally attributed to the ancient Greek poet Homer. The poem is the story of Odysseus, king of Ithaca, who wanders for 10 years (although the action of the poem covers only the final six weeks) trying to get home after the Trojan War. Learn more about the Odyssey. Discover Homer, the Greek poet known for The Iliad and The Odyssey. Explore his life, age, legacy, and his impact on literature and culture. Homer was a famous Greek poet known for epic tales like the Iliad and the Odyssey. Many think Homer was blind, and his name might mean 'blind' or 'captive' in ancient Greek. Homer's poems offer insights into Greek myth, religion, and the Trojan War, influenci

In [46]:
from langchain.tools import Tool
from bs4 import BeautifulSoup as bs

import requests
import json
import os

def get_vulnerability_report(input: str) -> str:
  """Search for vulnerability reports from CVE"""
  url = f"https://www.cve.org/CVERecord/SearchResults?query={input}"
  data = requests.get(url)
  soup = bs(data.text, 'html.parser')
  main = soup.find('main')
  return f"{soup}"

cve_info_tool = Tool(
    name="get_vulnerability_report",
    description="Grab today's vulnerability report from CVE",
    func=get_vulnerability_report
)

print(cve_info_tool.invoke("tor"))

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1.0" name="viewport"/>
<link href="/cvePurpleVFavicon.svg" rel="icon" type="image/x-icon"/>
<script src="https://cmp.osano.com/AzyhULTdPkqmy4aDN/46057d56-0263-4cca-abac-9adddada4f3b/osano.js"></script>
<script crossorigin="" src="/assets/index-IDpEvah4.js" type="module"></script>
<link crossorigin="" href="/assets/index-7af7oaXM.css" rel="stylesheet"/>
</head>
<body class="has-navbar-fixed-top">
<h1 hidden="">Common vulnerabilities and Exposures (CVE)</h1>
<noscript>
<strong>We're sorry but the CVE Website doesn't work properly without JavaScript enabled. Please enable it to continue.</strong>
</noscript>
<div id="app"></div>
</body>
</html>



In [47]:
from langchain.tools import Tool
from bs4 import BeautifulSoup as bs

def get_coincidences(input: str) -> str:
  """Search for possible PoC in public repositories"""
  url = f"https://github.com/search?q={input}"
  data = requests.get(url)
  soup = bs(data.text, 'html.parser')
  table = soup.find('table')
  return f"{soup}"

poc_tool = Tool(
    name="get_coincidences",
    description="Search for possible PoC in public repositories",
    func=get_coincidences
)

print(poc_tool.invoke("tor"))


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-c59dc71e3a4c.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/light_high_contrast-4bf0cb726930.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-89751e879

In [42]:
from typing import TypeDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.message import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

model = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct".
    huggingfacehub_api_token=f"{env_token}",
    top_k=10,
    top_p=0.95,
    temperature=0.01,
    repetition_penalty=1.02,
)

chat = ChatHuggingFace(llm=model, verbose=True)
tools = [search_tool, cve_info_tool, poc_tool]
chat_with_tools = chat.bind_tools(tools)

class AgentState(TypedDict):
  message: Annotated[List[AnyMessage], add_messages]

class assistant(state: AgentState):
  return {
      "messages": [chat_with_tools.invoke(state["messages"])],
  }

builder = StateGraph(AgentState)

builder.add_node("assistance", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_edge("tools", "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)

handler = builder.compile()

messages = [HumanMessage(content="Top ten vulnerabilities for tor")]
response = handler.invoke({"messages": messages})

print("👿:")
print(response['messages'][-1].content)

SyntaxError: invalid syntax (ipython-input-42-175283904.py, line 9)